# Library Importing

In [51]:
#Basic library
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

#Tensorflow
import tensorflow

#Sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df_items = pd.read_csv('beauty_amazon_items - beauty_amazon_items.csv')
df_iter = pd.read_csv('beauty_amazon_reviews - beauty_amazon_reviews.csv')

In [52]:
df_iter.shape

(253809, 6)

In [3]:
df_items.head()

,Description,Title,Brand,Price,ItemId
0,Loud 'N Clear Personal Sound Amplifier allows ...,Loud 'N Clear&trade; Personal Sound Amplifier,idea village,NaN,P4924
1,No7 Lift & Luminate Triple Action Serum 50ml b...,No7 Lift &amp; Luminate Triple Action Serum 50...,NaN,$44.99,P4622
2,No7 Stay Perfect Foundation now stays perfect ...,No7 Stay Perfect Foundation Cool Vanilla by No7,No7,$28.76,P6435
3,NaN,Wella Koleston Perfect Hair Colour 44/44 Mediu...,NaN,NaN,P4623
4,Lacto Calamine Skin Balance Daily Nourishing L...,Lacto Calamine Skin Balance Oil control 120 ml...,Pirmal Healthcare,$12.15,P7


In [4]:
df_iter.head()

,Rating,Time,UserId,ItemId,Review,Summary
0,1,"02 19, 2015",U0,P0,great,One Star
1,4,"12 18, 2014",U1,P0,My husband wanted to reading about the Negro ...,... to reading about the Negro Baseball and th...
2,4,"08 10, 2014",U2,P0,"This book was very informative, covering all a...",Worth the Read
3,5,"03 11, 2013",U3,P0,I am already a baseball fan and knew a bit abo...,Good Read
4,5,"12 25, 2011",U4,P0,This was a good story of the Black leagues. I ...,"More than facts, a good story read!"


# 1. Data Exploration

## 1.1 Item dataset

In [5]:
#Overview
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32890 entries, 0 to 32889
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Description  14581 non-null  object
 1   Title        32889 non-null  object
 2   Brand        17217 non-null  object
 3   Price        11459 non-null  object
 4   ItemId       32890 non-null  object
dtypes: object(5)
memory usage: 1.3+ MB


### 1.1.1 Missing value

In [6]:
#Missing value
df_items.isnull().sum()

Description    18309
Title              1
Brand          15673
Price          21431
ItemId             0
dtype: int64

In [7]:
df_items[df_items['Title'].isnull() == True]

,Description,Title,Brand,Price,ItemId
27016,NaN,NaN,BCW,$2.89 - $13.99,P3733


<p>In this project, only ItemId and Title is important. So we delete the row with missing Title.</p> <br/>
<b> However, if we delete the item, we have to delete all the interactions according to this item.</b>

### 1.1.2 Duplicate check

In [8]:
# Duplicates
df_items.duplicated().sum()

404

In [9]:
df_items[df_items['Description'] == 'Ultra (Box of 10) Corn Plane Blades']

,Description,Title,Brand,Price,ItemId
424,Ultra (Box of 10) Corn Plane Blades,Ultra (Box of 10) Corn Plane Blades,Ultra,$8.00,P4821
828,Ultra (Box of 10) Corn Plane Blades,Ultra (Box of 10) Corn Plane Blades,Ultra,$8.00,P4821


It varified the duplicates really exist!

## 1.2 Iteraction dataset

### 1.2.1 Missing value

In [10]:
#Missing value
df_iter.isnull().sum()

Rating       0
Time         0
UserId       0
ItemId       1
Review     230
Summary    130
dtype: int64

In [11]:
df_iter[df_iter['ItemId'].isnull() == True]

,Rating,Time,UserId,ItemId,Review,Summary
253808,5,"05 25, 2017",U21890,NaN,NaN,NaN


<p> In this case, we delete this single row because of the lack of ItemId.</p>

### 1.2.2 Duplicate check

In [12]:
# Duplicates
df_iter.duplicated().sum()

8717

In [13]:
df_iter[(df_iter['UserId'] == 'U12281') & (df_iter['ItemId'] == 'P62')]

,Rating,Time,UserId,ItemId,Review,Summary
12330,5,"03 09, 2016",U12281,P62,excellent,Five Stars
12331,5,"03 09, 2016",U12281,P62,excellent,Five Stars


There are 8717 duplicates and I will delete the **second** duplicates and remain the **first** ones.

# 2. Data Preprocessing

## 2.1 Missing values

In [14]:
# Delete the row of item dataset.
df_items_nomissing = df_items.dropna(subset=['Title'])

In [15]:
# Delete all rows with the ItemId of 'P3733' of interaction dataset.
df_iter_nomissing = df_iter[df_iter['ItemId'] != 'P3733']

In [16]:
#Delete all the missing value in iteraction dataset
df_iter_nomissing = df_iter_nomissing.dropna(subset=['ItemId'])

## 2.2 Duplicates

In [17]:
df_items.head()

,Description,Title,Brand,Price,ItemId
0,Loud 'N Clear Personal Sound Amplifier allows ...,Loud 'N Clear&trade; Personal Sound Amplifier,idea village,NaN,P4924
1,No7 Lift & Luminate Triple Action Serum 50ml b...,No7 Lift &amp; Luminate Triple Action Serum 50...,NaN,$44.99,P4622
2,No7 Stay Perfect Foundation now stays perfect ...,No7 Stay Perfect Foundation Cool Vanilla by No7,No7,$28.76,P6435
3,NaN,Wella Koleston Perfect Hair Colour 44/44 Mediu...,NaN,NaN,P4623
4,Lacto Calamine Skin Balance Daily Nourishing L...,Lacto Calamine Skin Balance Oil control 120 ml...,Pirmal Healthcare,$12.15,P7


In [18]:
#Drop the duplicates of items dataset and keep the first occurance.
df_items_no_duplicates = df_items_nomissing.drop_duplicates(inplace=False, keep="first", ignore_index=True)

In [19]:
#Drop the duplicates of iteraction dataset and keep the first occurance.
df_iter_no_duplicates = df_iter_nomissing.drop_duplicates(inplace=False, keep='first', ignore_index=True)

## 2.3 Data modifying

In [20]:
print(f'Length of items dataset:', len(df_items_no_duplicates))
print(f'Length of dataset has ItemId starting with "P":', len(df_items_no_duplicates['ItemId'][df_items_no_duplicates['ItemId'].str.startswith('P')]))
print('\n')
print(f'Length of iteraction dataset:', len(df_iter_no_duplicates))
print(f'Length of dataset has ItemId starting with "P":', len(df_iter_no_duplicates['ItemId'][df_iter_no_duplicates['ItemId'].str.startswith('P')]))


Length of items dataset: 32485
Length of dataset has ItemId starting with "P": 32485


Length of iteraction dataset: 245085
Length of dataset has ItemId starting with "P": 245085


I discovered that the Id of items all start with "P", some we can represent all ItemId with "P%". <br/> <br/>
In this case, we can modify all the ItemId from strings to numbers. <br/> <br/>
The same case for users.

In [21]:
#Change the "ItemId" column for item and iter dataset.
df_items_no_duplicates.loc[:, 'ItemId'] = df_items_no_duplicates.loc[:, 'ItemId'].str.replace('P', '', regex=True)
df_iter_no_duplicates.loc[:, 'ItemId'] = df_iter_no_duplicates.loc[:, 'ItemId'].str.replace('P', '', regex=True)

#Change the "UserId" column for iter dataset.
df_iter_no_duplicates.loc[:, 'UserId'] = df_iter_no_duplicates.loc[:, 'UserId'].str.replace('U', '', regex=True)

/var/folders/q7/h0qy2kmn1617pwg3_l5zhc7c0000gn/T/ipykernel_22470/1593752000.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_items_no_duplicates.loc[:, 'ItemId'] = df_items_no_duplicates.loc[:, 'ItemId'].str.replace('P', '', regex=True)
/var/folders/q7/h0qy2kmn1617pwg3_l5zhc7c0000gn/T/ipykernel_22470/1593752000.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_iter_no_duplicates.loc[:, 'ItemId'] = df_iter_no_duplicates.loc[:, 'ItemId'].str.replace('P', '', regex=True)
/var/folders/q7/h0qy2kmn1617

## 2.4 Data Splitting

In this case, we will split the iteration dataset into three parts, and the size of these three parts are as follows:
 - 70% training set;
 - 30% testing set.

First, we have to transfer the data type of the column **"Time"** to **"Datetime"**;
Then, reorder the dataset according to the **iteraction time**.

In [22]:
#Change data type.
df_iter_no_duplicates.loc[:, ('Time')] = pd.to_datetime(df_iter_no_duplicates.loc[:, ('Time')])

/var/folders/q7/h0qy2kmn1617pwg3_l5zhc7c0000gn/T/ipykernel_22470/2830399008.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_iter_no_duplicates.loc[:, ('Time')] = pd.to_datetime(df_iter_no_duplicates.loc[:, ('Time')])


In [23]:
#Sort the data type according to the Time column
df_iter_final = df_iter_no_duplicates.sort_values('Time', ascending=True, ignore_index=True)
df_items_final = df_items_no_duplicates.copy()

In [24]:
df_iter_final.duplicated().sum()

0

In [25]:
# Data Splitting on df_iter_final
X, y = df_iter_final.loc[:, ('Rating', 'Time', 'UserId', 'ItemId', 'Review')], df_iter_final.loc[:, 'ItemId'] # Add the summary column to avoid duplicates.

# Split for training and other datasets.
X_train, X_test, _, _ = train_test_split(X, y, train_size=0.7, shuffle=False)

In [26]:
# Split for training and other datasets.
X_train, X_test, _, _ = train_test_split(X, y, train_size=0.7, shuffle=False)

# 3. Recommendation Models

## 3.1 Popularity model

<h3>Datasets introduction:</h3>
 <li> Final processed datasets: df_items_final for items dataset and df_iter_final for iteraction dataset;</li>
 <li> X_train(val/test): splitted useful information datasets from iteraction dataset;</li>
 <li> y_train(val/test): recommendation validates datasets.</li>

In this model, we plan to **calculate the popularity of all items** and **rank them** according to the **popularity (average rating it reveived)**. <br/> <br/>
Then, we should form **personalized recommendation list** for each user (**exclude the objects they bought befor**). Then recommend them **the most popular product(s)**.

<h3>Important: here we only consider the average of items being purchased over 1000 times.</h3>

In [27]:
# Select out the popular items dataset
# Select the Items being purchased over 1000 times
a1_purchased_count = (X_train['ItemId'].value_counts())[X_train['ItemId'].value_counts()>=1000]
a1_filter_items = a1_purchased_count.index.to_list()
a1_filtered_dataset = X_train[X_train['ItemId'].isin(a1_filter_items)]

In [28]:
#Calculate the average scores of each item and rank them. Here we get the commending list globally.
a1_glo_recommend_dataset = a1_filtered_dataset.groupby('ItemId')['Rating'].mean().sort_values(ascending=False)
a1_glo_recommend_items = a1_glo_recommend_dataset.index.to_series()

In [29]:
#Then according to each user's historical behaviors, we exclude all the items have bought by each user, the recommend them the most popular item.
def a1_item_recommend(database, glo_recommend_items, userid):
    #Find out the ItemId of all items user has bought before.
    items_purchased = database[database['UserId'] == userid].ItemId.values
    
    #Filter out these items and return the recommendation list
    common_ele = glo_recommend_items[glo_recommend_items.isin(items_purchased)]
    
    recommendation = glo_recommend_items[~glo_recommend_items.isin(common_ele)][0]
    return userid, recommendation

In [30]:
# Run for all users, get the dictionary of recommendation for each user.
a1_dict = dict()
#####CHANGE THE USER NUMBER!!!
for userid in X_train['UserId'][:50]: #Because of the CPU problem, only select the first 5000 users.
    userid, a1_recommendation = a1_item_recommend(X_train, a1_glo_recommend_items, userid)
    a1_dict[userid] = a1_recommendation

In [31]:
# Validate and measure the recommendation accuracy:
a1_acc = list()
#####CHANGE THE USER NUMBER!!!
for userid in X_train['UserId'][:50]:
    purchased_after = X_test['ItemId'][X_test['UserId'] == userid]
#     print(purchased_after.values)
    if a1_dict[userid] in purchased_after.values:
        a1_acc.append(True)
    else:
        a1_acc.append(False)

print(f'This algorithm has a recommendation accuracy of:', round(a1_acc.count(True)/len(a1_acc) * 100, 1), '%')

This algorithm has a recommendation accuracy of: 0.0 %


## 3.2 Content-based model

In [32]:
# 计算user-profile (用购买物品的review计算)
# 计算item_feature 和 user-profile的cosine-similarity.
# 推荐用户cosine-similarity 最高的一个（五个）物品，并计算accuracy.

In [33]:
#Special process for Nan value in desciption column in items dataset and review column in X_train.
df_items_final['Description'] = df_items_final['Description'].fillna('None')
X_train['Review'] = X_train['Review'].fillna('None')

In [34]:
#Create feature vectors according to the description of objects.
#Instantiating Objects
tfidf = TfidfVectorizer(lowercase=True, stop_words='english')
item_features = tfidf.fit_transform(df_items_final['Description'])


In [35]:
#Calculate user-profile by using the reviews of them.
user_features = tfidf.transform(X_train['Review'])
user_profile = pd.DataFrame(user_features.toarray())

In [36]:
#Calculate the similarity matrix between items and users.
al2_cos = cosine_similarity(user_features, item_features)
np.fill_diagonal(al2_cos, 0)
al2_cos = pd.DataFrame(al2_cos)
al2_cos.head()

,0,1,2,3,4,5,6,7,8,9,...,32475,32476,32477,32478,32479,32480,32481,32482,32483,32484
0,0.000000,0.0,0.008746,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
1,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.030425,0.0
2,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.009466,0.0,0.00000,...,0.0,0.0,0.000000,0.0,0.068505,0.0,0.0,0.0,0.000000,0.0
3,0.028968,0.0,0.018417,0.0,0.0,0.0,0.0,0.029340,0.0,0.01625,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.006716,0.0
4,0.000000,0.0,0.030149,0.0,0.0,0.0,0.0,0.050917,0.0,0.00000,...,0.0,0.0,0.019821,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0


In [37]:
#Calculate the similarity matrix between items.
# al2_cos = cosine_similarity(item_features)
# np.fill_diagonal(al2_cos, 0)
# al2_cos = pd.DataFrame(al2_cos)
# al2_cos

In [38]:
# def a2_item_recommend(item_data, al2_cos, user_id, top_n=1):
#     #Get all rated items.
#     rated_items = X_train[X_train['UserId'] == user_id]['ItemId'].tolist()
    
    
#     #Calculate the similarity scores between rated products and other products.
#     item_scores=dict()
#     for item in rated_items:
#         similarity_scores = al2_cos[item].to_list()
#         rated_item_scores = np.array(X_train[X_train['ItemId'] == item]['Rating'])
#         weighted_scores = [x * y for x, y in zip(similarity_scores, rated_item_scores)]
#         item_scores.update(zip(item_data['ItemId'], weighted_scores))
#         print(item_scores)
        
#     #Delete all the items this user bought
#     for item in rated_items:
#         if item in item_scores.keys():
#             print('yes')
#             del item_scores[item]
            
#     #Recommend users with the item got the ighest score.
# #     recommendation = sorted(item_scores.items(), key=lambda x: x[1], reverse=True)[:top_n]
# #     return recommendation[0][0], 


In [39]:
# for user_id in X_train['UserId'][:1]:
#     recommendation = a2_item_recommend(df_items_final,al2_cos, user_id=1)
# #     print(f'User:', user_id, 'Item:', recommendation)

## 3.3 Collaborative filtering model

After analysing the structure of the two datasets: the amount of **user** is **much bigger** the amount of **item**.<br/><br/>
Thus, in this case, we plan to do **item-based memory** model.

In [40]:
X_train['ItemId'] = X_train['ItemId'].astype(int)
X_train['UserId'] = X_train['UserId'].astype(int)

In [41]:
#Create a pivot to set 'ItemId' as the index, 'UserId' as the column and "ratings" as values.
#Then fill all the Nan values with 0.
al3_pivot = X_train[['UserId', 'ItemId', 'Rating']].pivot_table(index='ItemId', columns='UserId', values='Rating').fillna(0).sort_index(axis=0, ascending=True)

In [42]:
al3_pivot

UserId,0,1,2,3,4,5,6,7,12,13,...,218889,218890,218891,218892,218893,218894,218895,218896,218897,218898
ItemId,,,,,,,,,,,,,,,,,,,,,
0,1.0,4.0,4.0,5.0,5.0,5.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4460,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4462,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
#Calculate the similarities between items.
al3_cos = cosine_similarity(al3_pivot)
np.fill_diagonal(al3_cos, 0)
#Turn it into a dataframe.
al3_cos = pd.DataFrame(al3_cos)

In [44]:
al3_cos

,0,1,2,3,4,5,6,7,8,9,...,4224,4225,4226,4227,4228,4229,4230,4231,4232,4233
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4229,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4230,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4231,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4232,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
al3_pivot.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            4447, 4449, 4451, 4456, 4459, 4460, 4462, 4463, 4464, 4465],
           dtype='int64', name='ItemId', length=4234)

In [46]:
X_train[X_train['UserId'] == 1]['ItemId'].tolist()

[0]

In [47]:
#Algorithm to recommend products to users.
def a3_item_recommend(al3_pivot, al3_cos, user_id, top_n=1):
    #Get the items this user rated.
    rated_items = X_train[X_train['UserId'] == user_id]['ItemId'].tolist()
    print(rated_items)
    #Calculate the similarity score between the items rated and other unrated items.
    item_scores = dict()
    for item in rated_items:
        sim_scores = al3_cos[item]
        rated_item_scores = al3_pivot[user_id]
        weighted_scores = sim_scores * rated_item_scores
        item_scores.update(zip(al3_pivot.index, weighted_scores))
    
    #Delete the items user has bought before.
    for item in rated_items:
        if item in item_scores:
            del item_scores[item]
    
    #Recommend the item to the user
    recommendation = sorted(item_scores.items(), key=lambda x: x[1], reverse=True)[:10]
    return recommendation

In [48]:
user_id = 15
a3_recommendation = a3_item_recommend(al3_pivot, al3_cos, user_id)


[1]


In [49]:
#Recommendation has come out, select the first recommendation and do a accuracy check.
a3_recommendation

[(0, 0.0),
 (2, 0.0),
 (3, 0.0),
 (4, 0.0),
 (5, 0.0),
 (6, 0.0),
 (7, 0.0),
 (8, 0.0),
 (9, 0.0),
 (10, 0.0)]

## 3.4 Model-based collaborative filtering model (Optional)

In [50]:
#使用surprise model

# 4. Models comparison

# 

## <h3 align="center"> © Haozhe TANG 07.2023. All rights reserved. <h3/>